In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# =====================================================
# Synthetic example (matching your Word documentation)
# =====================================================

# 4 חתולים (קבוצת מיעוט), 102 כלבים (קבוצת רוב)
# המודל "מוטה": מסווג הכל כ"כלב"
y_true = np.array([0] * 4 + [1] * 102)  # 0 = Cat, 1 = Dog
y_pred = np.array([1] * 106)  # המודל מסווג תמיד "כלב"

# ניצור גם הסתברויות סינתטיות
# חתולים: המודל "בטוח" שהם כלבים
# כלבים: המודל נותן הסתברות גבוהה
y_prob = np.concatenate(
    [
        np.random.uniform(0.6, 0.9, size=4),  # חתולים - תחזית שגויה ובטוחה
        np.random.uniform(0.8, 1.0, size=102),  # כלבים
    ]
)

group_cats = y_true == 0
group_dogs = y_true == 1

# =====================================================
# Helper functions (Fairness Metrics)
# =====================================================


def accuracy(true, pred, mask=None):
    if mask is None:
        return np.mean(true == pred)
    return np.mean(true[mask] == pred[mask])


def demographic_parity(pred, mask):
    """P(ŷ = 1 | group)"""
    return np.mean(pred[mask] == 1)


def tpr(true, pred, mask):
    """True Positive Rate"""
    tp = np.sum((true[mask] == 1) & (pred[mask] == 1))
    p = np.sum(true[mask] == 1)
    return tp / p if p > 0 else 0


def fpr(true, pred, mask):
    """False Positive Rate"""
    fp = np.sum((true[mask] == 0) & (pred[mask] == 1))
    n = np.sum(true[mask] == 0)
    return fp / n if n > 0 else 0


def disparate_impact(dp_minority, dp_majority):
    """DI = P(ŷ=1 | minority) / P(ŷ=1 | majority)"""
    return dp_minority / dp_majority if dp_majority > 0 else 0


def calibration_mean(prob, true, mask):
    """mean predicted probability + mean actual correctness"""
    return np.mean(prob[mask]), np.mean((prob[mask] > 0.5) == true[mask])


# =====================================================
# COMPUTE METRICS
# =====================================================

overall_acc = accuracy(y_true, y_pred)
acc_cats = accuracy(y_true, y_pred, group_cats)
acc_dogs = accuracy(y_true, y_pred, group_dogs)

dp_cats = demographic_parity(y_pred, group_cats)
dp_dogs = demographic_parity(y_pred, group_dogs)

tpr_cats = tpr(y_true, y_pred, group_cats)
tpr_dogs = tpr(y_true, y_pred, group_dogs)

fpr_cats = fpr(y_true, y_pred, group_cats)
fpr_dogs = fpr(y_true, y_pred, group_dogs)

di_score = disparate_impact(dp_cats, dp_dogs)

calib_cats = calibration_mean(y_prob, y_true, group_cats)
calib_dogs = calibration_mean(y_prob, y_true, group_dogs)

print("=== Fairness Metrics (Synthetic Example) ===")
print(f"Overall Accuracy: {overall_acc:.3f}")
print(f"Accuracy  - Cats: {acc_cats:.3f}, Dogs: {acc_dogs:.3f}")
print(f"DP        - Cats: {dp_cats:.3f}, Dogs: {dp_dogs:.3f}")
print(f"TPR       - Cats: {tpr_cats:.3f}, Dogs: {tpr_dogs:.3f}")
print(f"FPR       - Cats: {fpr_cats:.3f}, Dogs: {fpr_dogs:.3f}")
print(f"Disparate Impact: {di_score:.3f}")
print(f"Calibration (mean prob) - Cats: {calib_cats[0]:.3f}, Dogs: {calib_dogs[0]:.3f}")


# =====================================================
# BAR PLOT – Metrics per group
# =====================================================

metrics_labels = [
    "Accuracy",
    "Demographic Parity",
    "TPR (Equal Opportunity)",
    "FPR (Equalized Odds)",
    "Calibration (mean prob)",
]

cats_values = [acc_cats, dp_cats, tpr_cats, fpr_cats, calib_cats[0]]

dogs_values = [acc_dogs, dp_dogs, tpr_dogs, fpr_dogs, calib_dogs[0]]

x = np.arange(len(metrics_labels))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width / 2, cats_values, width, label="Cats (Minority)")
plt.bar(x + width / 2, dogs_values, width, label="Dogs (Majority)")

plt.xticks(x, metrics_labels, rotation=45, ha="right")
plt.ylabel("Value")
plt.title("Fairness Metrics Comparison (Synthetic Example)")
plt.ylim(0, 1.05)
plt.legend()
plt.tight_layout()
plt.show()